<a href="https://colab.research.google.com/github/MasaAsami/ml-dae/blob/main/notebooks/gBizinfoDEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from zipfile import ZipFile
import io
import sys
import os
import re
import pandas as pd

In [2]:
def loard_gbizinfo(file_id):
    res = requests.post(
        "https://info.gbiz.go.jp/hojin/Download",
        data={
            "downfile": file_id,  # ダウンロードしたいファイルの識別子と思われる
            "downtype": "zip",
            "downenc": "UTF-8",
        },
    )
    res.raise_for_status() #ここで例外処理

    zip_file = ZipFile(io.BytesIO(res.content))
    target_file = zip_file.namelist()[0]
    print(target_file)
    return pd.read_csv(zip_file.open(target_file))

base_df = loard_gbizinfo(7)

Kihonjoho_UTF-8.csv


In [3]:
finance_df = loard_gbizinfo(13)
null_ratio = finance_df.isnull().sum() / len(finance_df)
finance_df = finance_df.loc[:, null_ratio<0.5]
finance_df.sample(4).T

Zaimujoho_UTF-8.csv


<ipython-input-2-799bc58fad19>:15: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(zip_file.open(target_file))


,867,6222,16522,10105
法人番号,1010901002423,3020001025396,7010401103229,5010001000010
法人名（法人番号）,岡谷電機産業株式会社,ウインテスト株式会社,ＢＡＳＥ株式会社,ニチバン株式会社
本社所在地（法人番号）,東京都世田谷区等々力６丁目１６番９号,神奈川県横浜市西区平沼１丁目２番２４号,東京都港区六本木３丁目２番１号,東京都文京区関口２丁目３番３号
法人名,岡谷電機産業株式会社,ウインテスト株式会社,BASE株式会社,ニチバン株式会社
本社所在地,東京都世田谷区等々力六丁目16番９号,神奈川県横浜市西区平沼一丁目２番24号,東京都港区六本木三丁目２番１号,東京都文京区関口二丁目３番３号
会計基準,有価証券報告書を参照してください,有価証券報告書を参照してください,有価証券報告書を参照してください,有価証券報告書を参照してください
事業年度,第101期（自 2023年４月１日 至 2024年３月31日）,第32期（自 2024年１月１日 至 2024年12月31日）,第12期(自 2024年１月１日 至 2024年12月31日),第120期(自 2023年４月１日 至 2024年３月31日)
回次,0,2,2,1
売上高,12519229000.0,236025000.0,7635000000.0,43077000000.0
売上高（単位),JPY,JPY,JPY,JPY


In [4]:
def clean_year(x):
    try:
        return int(x)
    except:
        return int(1111)

finance_df["start_year"]= finance_df["事業年度"].apply(lambda x: str(x).split("年")[0][-4:]).apply(lambda x: clean_year(x))

id_cols = ['法人番号', '法人名（法人番号）', "start_year", '本社所在地（法人番号）', '法人名', '本社所在地', '会計基準', '事業年度','従業員数(単位)',  '売上高（単位)']
finance_df_g = finance_df.groupby(id_cols).agg(
    {
        '売上高': 'sum',
        '当期純利益又は当期純損失（△）': 'sum',
        '従業員数': 'mean',
        '資本金' : "mean",
        "純資産額": "mean"
    }
).reset_index().sort_values('法人番号', ascending=True).query("start_year == 2023")
assert finance_df_g.groupby("法人番号")["法人名（法人番号）"].count().max() == 1
assert len(finance_df_g["売上高（単位)"].unique()) == 1
assert len(finance_df_g["従業員数(単位)"].unique()) == 1

In [5]:
finance_df_g = finance_df_g.merge(
    base_df[
        [
            "法人番号", "設立年月日", "創業年", "郵便番号"
        ]
    ],
    on="法人番号"
    ,
    how="left"

)

In [6]:
finance_df_g["years_from_org"] = 2023 - pd.to_datetime(finance_df_g["設立年月日"]).dt.year
finance_df_g["prefecture"] = finance_df_g["本社所在地（法人番号）"].apply(lambda x:re.match('東京都|北海道|(?:京都|大阪)府|.{2,3}県', x).group())
pref_df_eda = finance_df_g.groupby("prefecture").agg(
    {
        "従業員数" : ["count", "mean", "std"]
    }
).sort_values(("従業員数", "count"), ascending=False).head(5)
display(pref_df_eda)
prefecture_list = pref_df_eda.index.to_list()
finance_df_g["prefecture"] = finance_df_g["prefecture"].apply(lambda x: x if x in prefecture_list else "others")

従業員数                          
           count         mean          std
prefecture                                
東京都         1460   928.329395  2441.461717
大阪府          347  1156.699232  3573.826678
愛知県          187  1449.559893  4176.351231
神奈川県         138  1324.557609  3811.647479
兵庫県           86   865.641860  2049.978586

In [7]:
finance_df_g = finance_df_g[
    [
        '法人番号', 'start_year', '法人名',
        '売上高', '当期純利益又は当期純損失（△）', '従業員数',
       '資本金', '純資産額', 'years_from_org', 'prefecture'
    ]
]

finance_df_g.columns = ['company_id', 'start_year', 'name', 'output_salse', 'output_profit', 'input_employee', 'input_capital', 'input_assets', 'years_from_org', 'prefecture']

In [8]:
finance_df_g.drop(columns=['company_id', 'start_year', "name", "prefecture"]).corr()

,output_salse,output_profit,input_employee,input_capital,input_assets,years_from_org
output_salse,1.000000,0.636427,0.739988,0.495983,0.717827,0.225090
output_profit,0.636427,1.000000,0.491256,0.454866,0.775938,0.169277
input_employee,0.739988,0.491256,1.000000,0.430987,0.653885,0.209308
input_capital,0.495983,0.454866,0.430987,1.000000,0.741127,0.181466
input_assets,0.717827,0.775938,0.653885,0.741127,1.000000,0.215527
years_from_org,0.225090,0.169277,0.209308,0.181466,0.215527,1.000000


In [9]:
finance_df_g = pd.concat(
    [
        finance_df_g.drop(columns=["prefecture"]),
        pd.get_dummies(finance_df_g["prefecture"], prefix='prefecture', drop_first=False).astype(int)
    ],
    axis=1
)

In [10]:
finance_df_g

,company_id,start_year,name,output_salse,output_profit,input_employee,input_capital,input_assets,years_from_org,prefecture_others,prefecture_兵庫県,prefecture_大阪府,prefecture_愛知県,prefecture_東京都,prefecture_神奈川県
0,1010001000006,2023,五洋建設株式会社,2.450596e+12,6.563000e+10,3114.2,3.044900e+10,1.375992e+11,73.0,0,0,0,0,1,0
1,1010001000708,2023,イチカワ株式会社,5.531100e+10,2.787000e+09,566.4,3.594000e+09,1.753240e+10,NaN,0,0,0,0,1,0
2,1010001003966,2023,スズデン株式会社,2.664722e+11,9.893661e+09,335.6,1.819230e+09,1.739748e+10,71.0,0,0,0,0,1,0
3,1010001008651,2023,ウシオ電機株式会社,2.869900e+11,5.419500e+10,1675.0,1.955600e+10,1.219060e+11,74.0,0,0,0,0,1,0
4,1010001008692,2023,住友商事株式会社,3.643391e+12,9.268130e+11,5137.0,2.199510e+11,1.289993e+12,104.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,9470001002353,2023,セーラー広告株式会社,1.130234e+10,-5.323500e+07,105.4,2.948680e+08,1.656468e+09,72.0,1,0,0,0,0,0
2906,9470001003062,2023,南海プライウッド株式会社,8.695385e+10,3.853405e+09,417.0,2.121000e+09,1.896820e+10,NaN,1,0,0,0,0,0
2907,9500001002341,2023,セキ株式会社,4.446798e+10,1.324849e+09,291.8,1.201700e+09,1.268480e+10,74.0,1,0,0,0,0,0
2908,9500001014345,2023,大王製紙株式会社,2.209288e+12,7.045500e+10,2548.2,5.179700e+10,1.938842e+11,NaN,1,0,0,0,0,0
